In [1]:
from simserver import SessionServer
import simserver
import pandas as pd
import psycopg2

In [2]:
# connect to database
dbname = 'podcast'
username = 'lindsay'
con = psycopg2.connect(database = dbname, user = username)
cursor = con.cursor()

In [3]:
# connect to servers
server_lsi = SessionServer('/tmp/simserver_lsi/')
server_lda = SessionServer('/tmp/simserver_lda/')
server_logentropy = SessionServer('/tmp/simserver_logentropy/')

In [33]:
# query for political gabfest
lsi_result = server_lsi.find_similar('15793')
lda_result = server_lda.find_similar('15793')
logentropy_result = server_logentropy.find_similar('15793')

In [34]:
lsi_result = pd.DataFrame(lsi_result, columns = ['id', 'similarity', 'null'])
lda_result = pd.DataFrame(lda_result, columns = ['id', 'similarity', 'null'])
logentropy_result = pd.DataFrame(logentropy_result, columns = ['id', 'similarity', 'null'])

In [35]:
del(lsi_result['null'])
del(lda_result['null'])
del(logentropy_result['null'])

In [36]:
query = """
SELECT id, name, raw_summary
FROM podcast
WHERE id IN %s;
"""
query = query.replace('\n', ' ')

In [37]:
def pg_int_array(the_list):
  return '(' + ','.join(the_list) + ')'

In [38]:
# run query for each model
cursor.execute(query % (pg_int_array(lsi_result['id'].tolist())))
lsi_query = cursor.fetchall()

cursor.execute(query % (pg_int_array(lda_result['id'].tolist())))
lda_query = cursor.fetchall()

cursor.execute(query % (pg_int_array(logentropy_result['id'].tolist())))
logentropy_query = cursor.fetchall()

In [39]:
# convert to df
lsi_query = pd.DataFrame(lsi_query, columns = ['id', 'name', 'raw_summary'])
lda_query = pd.DataFrame(lda_query, columns = ['id', 'name', 'raw_summary'])
logentropy_query = pd.DataFrame(logentropy_query, columns = ['id', 'name', 'raw_summary'])

In [40]:
# merge with similarity data
lsi_result['id'] = [int(x) for x in lsi_result['id']]
lda_result['id'] = [int(x) for x in lda_result['id']]
logentropy_result['id'] = [int(x) for x in logentropy_result['id']]

lsi_sim = pd.merge(lsi_result, lsi_query, how='inner', on='id')
lda_sim = pd.merge(lda_result, lda_query, how='inner', on='id')
logentropy_sim = pd.merge(logentropy_result, logentropy_query, how='inner', on='id')

In [61]:
def run_similarity(db_id, model):
    if model == "lsi":
        server = SessionServer('/tmp/simserver_lsi/')
    elif model == "lda":
        server = SessionServer('/tmp/simserver_lda/')
    elif model == "logentropy":
        server = SessionServer('/tmp/simserver_logentropy/')
    else:
        raise Exception("unknown model")
    
    sim_result = server.find_similar(db_id)
    sim_result = pd.DataFrame(sim_result, columns = ['id', 'similarity', 'null'])
    
    del(sim_result['null'])
    
    query = """
    SELECT id, name, raw_summary
    FROM podcast
    WHERE id IN %s;
    """
    query = query.replace('\n', ' ')
    
    cursor.execute(query % (pg_int_array(sim_result['id'].tolist())))
    query_result = cursor.fetchall()
    query_result = pd.DataFrame(query_result, columns = ['id', 'name', 'raw_summary'])
    
    sim_result['id'] = [int(x) for x in sim_result['id']]
    sim_final = pd.merge(sim_result, query_result, how='inner', on='id')
    
    return sim_final

In [64]:
run_similarity('15793', 'logentropy')

ERROR:sqlitedict:failed to delete /var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict2a732c
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/site-packages/sqlitedict.py", line 283, in terminate
    os.remove(self.filename)
OSError: [Errno 2] No such file or directory: '/var/folders/hg/kw83c_j57xqd055hr739mhvm0000gn/T/sqldict2a732c'


,id,similarity,name,raw_summary
0,15793,1.000000,Meowster Podcast,Hillarious podcast conversations about cats an...
1,17247,0.149550,Smart People Podcast | Podcast interviews feat...,Smart People Podcast is simply a place to hear...
2,14051,0.149465,Dirty Sons of Pitches,"Every week, Nate, Ben, ad Eric, with the occas..."
3,17272,0.148960,SOFA DOGS Podcast,A small band of pop culture enthusiasts (geeks...
4,16381,0.146597,Oh Behave - Harmony in the household with your...,These pet podcasts are all about achieving har...
5,13950,0.143389,Dark Discussions Podcast,WWW.DARKDISCUSSIONS.COM - your place for the d...
6,17770,0.142581,"The Toadcast - the weekly podcast from Song, b...",Every couple of weeks I sit down with a couple...
7,16282,0.140216,2 Boomer Broads,2 Boomer Broads – Life – Love – Laughs &amp; U...
8,18019,0.139064,TV Guidance Counselor Podcast,Each week Boston Comedian Ken Reid and his gue...
9,18593,0.138962,The Art of Charm | Confidence | Relationship &...,"The Art of Charm Podcast is where curious, hun..."
